# 온라인 뉴스 데이터 수집

웹사이트의 데이터가 언제나 CSV나 JSON과 같은 편안한 형식으로 제공되지 않는다.

그러나, HTML 데이터는 구조화된 데이터이므로, 웹 스크래핑을 통해 데이터를 수집할 수 있다.

### 웹 스크래핑
- 컴퓨터 프로그램을 통해 웹 페이지를 탐색하고 필요한 데이터를 유용한 형식에 맞춰 수집하는 작업
- 크롤링이라고도 부름
- 웹 스크래핑을 명시적으로 금지하는 경우도 많으며, API를 통해서만 데이터를 제공하는 경우도 많다
    - 사이트의 이용약관을 확인하자

# 네이버 뉴스 데이터 수집

- http://news.naver.com/
- http://news.naver.com/main/search/search.nhn?
- 사이트 구조와 html 구조를 이해하면 쉽게 가져올 수 있다!

In [ ]:
from bs4 import BeautifulSoup as bsoup
import requests
from urllib.parse import quote

In [ ]:
def get_url(qword, sdate, edate, page):
    """
    description: 뉴스 검색 결과 url을 가져오는 함수
    
    input: 검색 키워드 문자열
           뉴스 기간 시작일과 종료일
           검색 결과의 페이지 번호
    output: 검색 결과 url
    """
    
    base_url = (
        'http://news.naver.com/main/search/search.nhn?'
        'query={}&startDate={}&endDate={}&page={}'
    )
    # '문재인' --> '%B9%AE%C0%E7%C0%CE'
    qword = quote(qword, encoding='MS949') #EUC-KR OK
    return_url = base_url.format(qword, sdate, edate, page)
    return return_url

In [ ]:
my_url = get_url("문재인", '2017-07-14', '2017-07-14', 1)
my_url

In [ ]:
def get_html(url):
    """
    description: url에 해당하는 html 문자열을 가져옴
    
    input: url
    output: html 문자열    
    """
    page_html = requests.get(url).text
    return page_html

In [ ]:
my_html = get_html(my_url)
my_html

In [ ]:
def get_go_naver_urls(page_html):
    # beautifulsoup documentation: 
    # https://www.crummy.com/software/BeautifulSoup/bs4/doc/
    soup = bsoup(page_html, 'html.parser')
    href_list = soup.find_all('a', class_='go_naver')
    href_list = [href.get('href') for href in href_list]
    return href_list

In [ ]:
my_urls = get_go_naver_urls(my_html)
my_urls

In [ ]:
def get_news(url):
    """
    description: 네이버 뉴스 기사 url을 받아서, 필요한 내용만 추출하여 딕셔너리로 반환
    
    input: 네이버 뉴스 기사 url
    output: {뉴스기사 제목, 내용, 언론사, 일시}로 구성된 딕셔너리
    """
    page_html = get_html(url)
    soup = bsoup(page_html, 'html.parser')
    title = soup.find('h3', id='articleTitle')
    if title:
        title = title.text
        press = soup.find('div', class_="press_logo")
        press = press.img.get('title')
        dt = soup.find('span', class_='t11').text
        article = soup.find('div', id='articleBodyContents').text
        news_dic = {
            'title': title,
            'press': press,
            'datetime': dt,
            'article': article
        }
        return news_dic
    else:
        return None

In [ ]:
for url in my_urls:
    my_dict = get_news(url)
    print(my_dict)
    print('-' * 20)

### 예시

In [ ]:
qword = '문재인' #%B9%AE%C0%E7%C0%CE
sdate = '2017-07-14'
edate = '2017-07-14'
page = 1

url = get_url(qword, sdate, edate, page)
print(url)
page_html = get_html(url)
href_list = get_go_naver_urls(page_html)
for href in href_list:
    print(href)
    news = get_news(href)
    if news:
        print(get_news(href))